In [6]:
# !pip install transformers
# !pip install datasets
# !pip install sklearn
# !pip install emtions
# !pip install ipywidgets
# !pip install emojis
# !pip3 install umap-learn

In [7]:
import sys
sys.path.append('../assets/')

In [8]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /home/abdelrahman/.huggingface/token


In [10]:
# Main libraries
from data_shuffling_split import *
from preprocess_text import *
from datasets import load_dataset, list_datasets, Dataset, DatasetDict, ClassLabel
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification
import matplotlib.pyplot as plt
import torch
from sklearn.dummy import DummyClassifier
import numpy as np
from umap import UMAP
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments

In [11]:
dialect_datasets = load_dataset('Abdelrahman-Rezk/Arabic_Dialect_Identification', use_auth_token=True)
dialect_datasets

Using custom data configuration Abdelrahman-Rezk--Arabic_Dialect_Identification-256d82f412398eec
Reusing dataset parquet (/home/abdelrahman/.cache/huggingface/datasets/Abdelrahman-Rezk___parquet/Abdelrahman-Rezk--Arabic_Dialect_Identification-256d82f412398eec/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['id', 'label', 'text'],
        num_rows: 9164
    })
    train: Dataset({
        features: ['id', 'label', 'text'],
        num_rows: 440052
    })
    test: Dataset({
        features: ['id', 'label', 'text'],
        num_rows: 8981
    })
})

In [12]:
train_data = dialect_datasets['train']
print(train_data.column_names)
print("="*50)
print(train_data[0])
print("="*50)
print(train_data.features)
print("="*50)
print(train_data[:3])

['id', 'label', 'text']
{'id': 1046024946705735552, 'label': 9, 'text': '@Ahmed_Hamza27 اتمنى صراحة و خصوصا #طال_عمره بينا وبينه قصة عشق قديمة😍😍'}
{'id': Value(dtype='int64', id=None), 'label': ClassLabel(num_classes=18, names=['OM', 'SD', 'SA', 'KW', 'QA', 'LB', 'JO', 'SY', 'IQ', 'MA', 'EG', 'PL', 'YE', 'BH', 'DZ', 'AE', 'TN', 'LY'], id=None), 'text': Value(dtype='string', id=None)}
{'id': [1046024946705735552, 1140189387508134016, 1051416181855440768], 'label': [9, 17, 0], 'text': ['@Ahmed_Hamza27 اتمنى صراحة و خصوصا #طال_عمره بينا وبينه قصة عشق قديمة😍😍', '@rJiM4CnIFTgml9g @zamnissi طبعا عرفته من اللي بشعار الليڤر', '@OmanisFollowers @ooredoo @TRA_OMAN @motc_om هيه هيه م يتغير شي.. كلهم ع بعضهم متولفين😪.. بيجيك مدير فرع قريب منك عشان يتساعد معك عشان بس يثبت جدارته لوظيفيه فكرسيه ويحاول يتعاون معك لكن فالنهايه بترجع ع نفس المشكله والدائرة تدور لا بيقدموا ولا بيأخروا😪😪.. وحدي غاسله ايدي منهم']}


In [ ]:
train_data.set_format(type="pandas")
df = train_data[:]
df.head()

In [ ]:
def label_int2str(row):
    return train_data.features['label'].int2str(row)
df['label_name'] = df['label'].apply(label_int2str)
df.head()

In [ ]:
train_data

In [ ]:
df['label_name'].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes")
plt.show()

In [ ]:
df['tokens per tweet'] = df['text'].str.split().apply(len)
df.boxplot("tokens per tweet", by="label_name", grid=False, showfliers=False, color="black")
plt.suptitle("")
plt.xlabel("")
plt.show()

In [ ]:
train_data.reset_format()

## From Text to Tokens

- تحديد انهى موديلز هنستخدمها
- تحديد التكوينزر الخاص بكل موديل
- التجريب على بعض الأمثلة من الداتا

https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-da

https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-msa

https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-msa-did-madar-twitter5?text=%D8%B9%D8%A7%D9%85%D9%84+%D8%A7%D9%8A%D9%87+%D8%9F

Edit tokenizer in our model to be limited to 124 word !?

## Subword Tokenization
The basic idea behind subword tokenization is to combine the best aspects of character and word tokenization. On the one hand, we want to split rare words into smaller units to allow the model to deal with complex words and misspellings.

In [ ]:
model_ckpt = "CAMeL-Lab/bert-base-arabic-camelbert-msa"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
some_text = train_data['text'][8]
some_text

In [ ]:
encoded_text = tokenizer(some_text)
print(encoded_text)

In [ ]:
print(tokenizer.convert_ids_to_tokens(encoded_text['input_ids']))
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded_text['input_ids'])))

In [ ]:
print(tokenizer.vocab_size)
print(tokenizer.model_max_length)
print(tokenizer.model_input_names)

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

print(tokenize(train_data[:2]))

In [ ]:
batch = {}
batch['text'] = "الهدف من الحياة هو سث[MASK] . "
encoded_text = tokenize(batch)
print(encoded_text)
print(tokenizer.convert_ids_to_tokens(encoded_text['input_ids']))

### Model Special Token & ITs ids

In [ ]:
print(tokenizer.pad_token, tokenizer.pad_token_id)
print(tokenizer.unk_token, tokenizer.unk_token_id)
print(tokenizer.cls_token, tokenizer.cls_token_id)
print(tokenizer.sep_token, tokenizer.sep_token_id)
print(tokenizer.mask_token, tokenizer.mask_token_id)

In [ ]:
# batch_size=None to get the length inputs based on the large tweet
dialect_datasets_encoded = dialect_datasets.map(tokenize, batched=True, batch_size=1024)

In [ ]:
print(dialect_datasets_encoded.column_names)

In [ ]:
# The AutoModel class converts the token encodings to embeddings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [ ]:
device

In [ ]:
inputs = tokenizer(some_text, return_tensors="pt")
print(f"Input tensor shape: {inputs['input_ids'].size()}")

In [ ]:
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
# [batch_size, n_tokens, hidden_dim]
print(outputs.last_hidden_state.shape)

In [ ]:
tokenizer.model_input_names

In [ ]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    # print("="*50)
    # print(tokenizer.model_input_names)
    # print(batch.items())
    inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
    
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [ ]:
dialect_datasets_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label", "token_type_ids"])

In [ ]:
dialect_datasets_hidden = dialect_datasets_encoded.map(extract_hidden_states, batched=True,  batch_size=1024)

In [ ]:
dialect_datasets_hidden["train"].column_names

In [ ]:
# Scale features to [0,1] range
X_scaled = MinMaxScaler().fit_transform(X_train)
# Initialize and fit UMAP
mapper = UMAP(n_components=2, metric="cosine").fit(X_scaled)
# Create a DataFrame of 2D embeddings
df_emb = pd.DataFrame(mapper.embedding_, columns=["X", "Y"])
df_emb["label"] = y_train
df_emb.head()

In [ ]:
# fig, axes = plt.subplots(2, 3, figsize=(7,5))
# axes = axes.flatten()
# cmaps = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
#                       'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
#                       'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
# labels = dialect_datasets["train"].features["label"].names

# for i, (label, cmap) in enumerate(zip(labels, cmaps)):
#     df_emb_sub = df_emb.query(f"label == {i}")
#     axes[i].hexbin(df_emb_sub["X"], df_emb_sub["Y"], cmap=cmap, gridsize=20, linewidths=(0,))

In [ ]:
num_labels = len(set(train_data['label']))
model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

In [ ]:
def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()
    


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids # acutall
    preds = pred.predictions.argmax(-1) # predicted
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
batch_size = 256
logging_steps = len(dialect_datasets["train"]) // batch_size

model_name = f"{model_ckpt}-finetuned-Arabic_Dialect_Identification_model_1"
training_args = TrainingArguments(output_dir=model_name,
                                num_train_epochs=5,
                                learning_rate=2e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                evaluation_strategy="epoch",
                                disable_tqdm=False,
                                logging_steps=logging_steps,
                                push_to_hub=True,
                                log_level="error")

In [ ]:
trainer = Trainer(model=model, args=training_args,
                compute_metrics=compute_metrics,
                train_dataset=dialect_datasets["train"],
                eval_dataset=dialect_datasets["validation"],
                tokenizer=tokenizer)
trainer.train()

In [ ]:
preds_output = trainer.predict(dialect_datasets["validation"])
print(preds_output)
y_preds = np.argmax(preds_output.predictions, axis=1)
y_valid = dialect_datasets["validation"]['label']
labels = dialect_datasets["train"].features["label"].names
plot_confusion_matrix(y_preds, y_valid, labels)